## BERTopic

### Bibliotecas

In [1]:
#Bibliotecas

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import numpy as np
import json
import glob
import nltk
import re
from transformers import pipeline
from sklearn.cluster import KMeans

#Estilizar conteúdo
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

def estilo_tabelas(df, max_altura='300px', casas_decimais=2):
    return (
        df.style.set_table_styles(
            [
                {'selector': 'thead th', 'props': [('font-size', '12px'), ('text-align', 'center'), ('border-bottom', '2px solid #007BFF')]},
                {'selector': 'td', 'props': [('font-size', '10px'), ('text-align', 'center'), ('max-height', '40px'), ('white-space', 'nowrap'), ('text-overflow', 'ellipsis'), ('overflow', 'hidden'), ('max-width', '100px')]},
                {'selector': 'tr:nth-child(odd)', 'props': [('background-color', '#ffffff')]},
                {'selector': 'table', 'props': [('width', '90%'), ('margin-left', 'auto'), ('margin-right', 'auto'), ('border-collapse', 'collapse')]},
                {'selector': 'td, th', 'props': [('border', '1px solid #666')]},
            ]
        ).set_properties(
            **{'border-color': 'darkgray', 'border-style': 'solid', 'border-width': '1px'}
        ).set_table_attributes(
            f'style="height:auto; overflow:auto; max-height:{max_altura}; display:block;"'  
        ).format(
            precision=casas_decimais  
        )
    )

ModuleNotFoundError: No module named 'nltk'

### Base de dados

In [4]:
object_columns = ['Protocolo_S2iD', 'Nome_Municipio', 'Sigla_UF', 'regiao',
                  'Setores Censitários', 'Status', 'DH_Descricao', 'DM_Descricao',
                  'DA_Descricao', 'DA_Polui/cont da água', 'DA_Polui/cont do ar',
                  'DA_Polui/cont do solo', 'DA_Dimi/exauri hídrico',
                  "DA_Incêndi parques/APA's/APP's", 'PEPL_Descricao', 'PEPR_Descricao',
                  'Categoria', 'Grupo', 'Subgrupo', 'Tipo', 'Subtipo']

dtype = {col: 'object' for col in object_columns}

df_eventos = pd.read_csv(
    "https://raw.githubusercontent.com/brunagmoura/PrevisorReconhecimento/refs/heads/main/df_eventos_desastres_rec_nrec.csv",
    sep=';',
    decimal=',',
    dtype=dtype)

colunas_descr = [ 'Status', 'DM_Descricao',
    'DA_Descricao',
    'PEPL_Descricao',
    'PEPR_Descricao',
    'DH_Descricao' ]

df_eventos_descr = df_eventos.copy()
df_eventos_descr = df_eventos_descr[colunas_descr]
df_eventos_descr = df_eventos_descr.fillna('')
df_eventos_descr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26258 entries, 0 to 26257
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Status          26258 non-null  object
 1   DM_Descricao    26258 non-null  object
 2   DA_Descricao    26258 non-null  object
 3   PEPL_Descricao  26258 non-null  object
 4   PEPR_Descricao  26258 non-null  object
 5   DH_Descricao    26258 non-null  object
dtypes: object(6)
memory usage: 1.2+ MB


In [5]:
# Filtra o DataFrame para separar os eventos "Reconhecidos" e "Não reconhecidos"

df_reconhecido = df_eventos_descr[df_eventos_descr['Status'] == 'Reconhecido']
lista_descr_reconhecido = df_reconhecido[colunas_descr].values.tolist()

df_nao_reconhecido = df_eventos_descr[df_eventos_descr['Status'] == 'Não reconhecido']
lista_descr_nao_rec = df_nao_reconhecido[colunas_descr].values.tolist()

lista_dm = df_eventos_descr['DM_Descricao'].values.tolist()
lista_da = df_eventos_descr['DA_Descricao'].values.tolist()
lista_pepl = df_eventos_descr['PEPL_Descricao'].values.tolist()
lista_pepr = df_eventos_descr['PEPR_Descricao'].values.tolist()
lista_dh = df_eventos_descr['DH_Descricao'].values.tolist()

lista_dm_r = df_reconhecido['DM_Descricao'].values.tolist()
lista_da_r = df_reconhecido['DA_Descricao'].values.tolist()
lista_pepl_r = df_reconhecido['PEPL_Descricao'].values.tolist()
lista_pepr_r = df_reconhecido['PEPR_Descricao'].values.tolist()
lista_dh_r = df_reconhecido['DH_Descricao'].values.tolist()

lista_dm_n = df_nao_reconhecido['DM_Descricao'].values.tolist()
lista_da_n = df_nao_reconhecido['DA_Descricao'].values.tolist()
lista_pepl_n = df_nao_reconhecido['PEPL_Descricao'].values.tolist()
lista_pepr_n = df_nao_reconhecido['PEPR_Descricao'].values.tolist()
lista_dh_n = df_nao_reconhecido['DH_Descricao'].values.tolist()

## Imprime o resultado
print('Número de reconhecidos:')
print(len(lista_descr_reconhecido))
print('\nNúmero de não reconhecidos:')
print(len(lista_descr_nao_rec))

Número de reconhecidos:
23871

Número de não reconhecidos:
2387


In [9]:
#Transforma as descrições em lista
lista_tudo = sum([lista_dm, lista_da, lista_pepl, lista_pepr, lista_dh], [])
lista_rec_ = sum([lista_dm_r, lista_da_r, lista_pepl_r, lista_pepr_r, lista_dh_r], [])
lista_nao_ = sum([lista_dm_n, lista_da_n, lista_pepl_n, lista_pepr_n, lista_dh_n], [])

padrao = r"^$"

lista_rec = [string for string in lista_rec_ if not re.search(padrao, string)]
print(lista_rec_[0:10])
lista_nao = [string for string in lista_nao_ if not re.search(padrao, string)]
print(lista_nao_[0:10])

['', '', '', '', '', '', '', '', '', '']
['', 'NIHIL', '', 'Residencias particulares que necessitam de reparos, descontaminação e pintura. Móveis e utensílios danificados ou perdidos.', 'houve deslizamento, desbarrancamento, deslocamento da Rede Pluvial, oferendo risco às casas localizadas nas proximidades de 1,5 m (um metro e cinquenta centímetros) de distância, com cota de piso da residência a 3,5 m (três metros e cinquenta centímetros) de diferença de nível da referida avenida. Também há risco de queda de veículos sobre o telhado de cobertura das residências localizadas no lado direito ao longo da avenida; sendo necessário a construção urgente de um muro de arrimo e substituição de pavimento.', 'Destruição do de galeria de agua pluviais em tubo armico e ferro galvanizado corrugado de 2 m de diametro, e 02 adutores de água em PVC (DeFoFo) de 4 pol.', '- Nas Ruas Joaquim Ribeiro de Carvalho, Deputado Manoel Costa e Francisco Rodrigues Coura houve deslizamentos de encostas e necessita 

### Modelo BERT

#### Dados para rodar o modelo BERT

In [ ]:
# Carregar o pipeline de feature-extraction com BERT
feature_extractor = pipeline("feature-extraction", model="bert-base-uncased", tokenizer="bert-base-uncased")

# Usando lista_tudo (descrições combinadas)
# Gera embeddings para cada descrição em lista_tudo
embeddings = feature_extractor(lista_tudo, padding=True, truncation=True, return_tensors="pt")

# Converta os embeddings para numpy para serem usados em KMeans ou outro agrupamento
embeddings_np = np.array(embeddings)

# Verifique o tamanho dos embeddings
print(embeddings_np.shape)

# Definir o número de tópicos que queremos, exemplo: 5
n_clusters = 5

# Executar o K-Means nos embeddings
kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(embeddings_np)

# Ver os rótulos (tópicos) atribuídos a cada descrição
labels = kmeans.labels_

# Ver os resultados de tópicos atribuídos às descrições
for i, label in enumerate(labels):
    print(f"Descrição: {lista_tudo[i]} | Tópico: {label}")

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
